# DRS Financial Impact Model – Interactive Notebook (v3)

This notebook models the financial impact of Arizet's DRS system on your prop firm.

You can:
- Adjust **all baseline business assumptions** (challenges sold, fee, payout ratio, CAC, LTV, etc.).
- Adjust **all DRS economics** (avg rescue price, revenue-share %, etc.).
- Adjust **Monte Carlo ranges** for trader behavior (adoption, rescues per adopter, cannibalization, incremental payouts).
- Run Monte Carlo simulations and see how DRS affects:
  - Payout ratio
  - Net profit per cohort
  - LTV profit per buyer
  - Probability that DRS is a "good deal" (payout ratio within target band AND LTV profit ≥ baseline).

At the bottom of the interactive section you also get a **verdict** text that updates as you move the sliders.

### Exporting for execs
- If you export this notebook as **HTML** (File → Download as → HTML), all charts and current results are saved, but the sliders will **not** recompute simulations in a standalone browser – Python is needed for that.
- For truly live web interaction without Jupyter, you’d host this with something like **Voilà** or another Jupyter server.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

try:
    import ipywidgets as widgets
    from ipywidgets import VBox, HBox, Layout
    HAS_WIDGETS = True
except ImportError:
    HAS_WIDGETS = False
    print('ipywidgets is not installed. Install with `pip install ipywidgets` for full interactivity.')


In [2]:
def drs_outcome(
    N_challenges=1000,
    challenge_fee=549.0,
    baseline_payout_ratio=0.40,
    cac_per_challenge=27.5,
    avg_ltv_challenges=2.0,
    avg_rescue_price=350.0,
    adoption_rate=0.25,
    rescues_per_adopter=2.0,
    cannibalization_rate=0.30,
    extra_payout_per_drs_rev=0.30,
    rev_share_rate=0.40,
):
    """Single deterministic scenario: returns baseline + with-DRS metrics."""
    # Baseline economics (no DRS)
    baseline_challenge_revenue = N_challenges * challenge_fee
    baseline_payouts = baseline_challenge_revenue * baseline_payout_ratio
    baseline_cac = N_challenges * cac_per_challenge
    baseline_net_profit = baseline_challenge_revenue - baseline_payouts - baseline_cac
    baseline_buyers = N_challenges / avg_ltv_challenges if avg_ltv_challenges > 0 else np.nan
    baseline_ltv_profit = baseline_net_profit / baseline_buyers if baseline_buyers > 0 else np.nan

    # DRS revenue
    adopters = N_challenges * adoption_rate
    total_rescues = adopters * rescues_per_adopter
    drs_revenue = total_rescues * avg_rescue_price

    # Cannibalization: some DRS revenue replaces challenge revenue
    lost_challenge_revenue = cannibalization_rate * drs_revenue
    challenge_rev_after = baseline_challenge_revenue - lost_challenge_revenue
    total_revenue = challenge_rev_after + drs_revenue

    # Payouts
    payouts_from_challenge = challenge_rev_after * baseline_payout_ratio
    incremental_payouts_from_drs = drs_revenue * extra_payout_per_drs_rev
    total_payouts = payouts_from_challenge + incremental_payouts_from_drs

    # CAC (unchanged per cohort)
    cac = baseline_cac

    # Arizet share on net DRS margin after incremental payouts
    net_drs_margin = max(0.0, drs_revenue - incremental_payouts_from_drs)
    arizet_fee = rev_share_rate * net_drs_margin

    net_profit = total_revenue - total_payouts - cac - arizet_fee
    ltv_profit = net_profit / baseline_buyers if baseline_buyers > 0 else np.nan
    payout_ratio = total_payouts / total_revenue if total_revenue > 0 else np.nan

    return {
        'baseline_challenge_revenue': baseline_challenge_revenue,
        'baseline_payouts': baseline_payouts,
        'baseline_cac': baseline_cac,
        'baseline_net_profit': baseline_net_profit,
        'baseline_buyers': baseline_buyers,
        'baseline_ltv_profit': baseline_ltv_profit,
        'drs_revenue': drs_revenue,
        'lost_challenge_revenue': lost_challenge_revenue,
        'challenge_rev_after': challenge_rev_after,
        'total_revenue': total_revenue,
        'payouts_from_challenge': payouts_from_challenge,
        'incremental_payouts_from_drs': incremental_payouts_from_drs,
        'total_payouts': total_payouts,
        'cac': cac,
        'arizet_fee': arizet_fee,
        'net_profit': net_profit,
        'ltv_profit': ltv_profit,
        'payout_ratio': payout_ratio,
    }

def sample_triangular(low, mode, high, size=None):
    """Triangular distribution for intuitive min/mode/max ranges."""
    return np.random.triangular(low, mode, high, size=size)

def simulate_drs(
    n_sims=5000,
    N_challenges=1000,
    challenge_fee=549.0,
    baseline_payout_ratio=0.40,
    cac_per_challenge=27.5,
    avg_ltv_challenges=2.0,
    avg_rescue_price=350.0,
    target_max_payout_ratio=0.40,
    adoption_range=(0.05, 0.20, 0.35),
    rescues_range=(1.0, 2.0, 4.0),
    cannibal_range=(0.10, 0.30, 0.70),
    extra_payout_range=(0.10, 0.30, 0.60),
    rev_share_rate=0.40,
):
    """Monte Carlo over uncertain DRS behavior.
    Returns (df, summary).
    """
    base = drs_outcome(
        N_challenges=N_challenges,
        challenge_fee=challenge_fee,
        baseline_payout_ratio=baseline_payout_ratio,
        cac_per_challenge=cac_per_challenge,
        avg_ltv_challenges=avg_ltv_challenges,
        avg_rescue_price=avg_rescue_price,
        adoption_rate=0.0,
        rescues_per_adopter=0.0,
        cannibalization_rate=0.0,
        extra_payout_per_drs_rev=0.0,
        rev_share_rate=0.0,
    )
    baseline_net = base['baseline_net_profit']
    baseline_ltv = base['baseline_ltv_profit']

    adoption_samples = sample_triangular(*adoption_range, size=n_sims)
    rescues_samples = sample_triangular(*rescues_range, size=n_sims)
    cannibal_samples = sample_triangular(*cannibal_range, size=n_sims)
    extra_payout_samples = sample_triangular(*extra_payout_range, size=n_sims)

    records = []
    for i in range(n_sims):
        res = drs_outcome(
            N_challenges=N_challenges,
            challenge_fee=challenge_fee,
            baseline_payout_ratio=baseline_payout_ratio,
            cac_per_challenge=cac_per_challenge,
            avg_ltv_challenges=avg_ltv_challenges,
            avg_rescue_price=avg_rescue_price,
            adoption_rate=float(adoption_samples[i]),
            rescues_per_adopter=float(rescues_samples[i]),
            cannibalization_rate=float(cannibal_samples[i]),
            extra_payout_per_drs_rev=float(extra_payout_samples[i]),
            rev_share_rate=rev_share_rate,
        )
        res['adoption_rate'] = float(adoption_samples[i])
        res['rescues_per_adopter'] = float(rescues_samples[i])
        res['cannibalization_rate'] = float(cannibal_samples[i])
        res['extra_payout_per_drs_rev'] = float(extra_payout_samples[i])
        res['delta_net_profit'] = res['net_profit'] - baseline_net
        res['delta_ltv_profit'] = res['ltv_profit'] - baseline_ltv
        res['good_deal'] = int((res['payout_ratio'] <= target_max_payout_ratio) and (res['ltv_profit'] >= baseline_ltv))
        records.append(res)

    df = pd.DataFrame(records)
    summary = {
        'baseline_net_profit': baseline_net,
        'baseline_ltv_profit': baseline_ltv,
        'mean_delta_net_profit': df['delta_net_profit'].mean(),
        'median_delta_net_profit': df['delta_net_profit'].median(),
        'prob_good_deal': df['good_deal'].mean(),
        'mean_payout_ratio': df['payout_ratio'].mean(),
        'p5_payout_ratio': df['payout_ratio'].quantile(0.05),
        'p95_payout_ratio': df['payout_ratio'].quantile(0.95),
        'mean_ltv_profit': df['ltv_profit'].mean(),
    }
    return df, summary

def revenue_share_sweep(share_grid, n_sims=2000, **kwargs):
    rows = []
    for share in share_grid:
        df_tmp, summ = simulate_drs(n_sims=n_sims, rev_share_rate=share, **kwargs)
        rows.append({
            'rev_share_rate': share,
            'mean_delta_net_profit': summ['mean_delta_net_profit'],
            'prob_good_deal': summ['prob_good_deal'],
            'mean_payout_ratio': summ['mean_payout_ratio'],
        })
    return pd.DataFrame(rows)


In [3]:
if not HAS_WIDGETS:
    print('ipywidgets not available. Install it and re-run this cell for interactivity.')
else:
    # Baseline business widgets
    N_challenges_w = widgets.IntText(value=1000, description='Challenges', layout=Layout(width='200px'))
    challenge_fee_w = widgets.FloatText(value=549.0, description='Fee', layout=Layout(width='200px'))
    baseline_payout_ratio_w = widgets.FloatSlider(value=0.40, min=0.1, max=0.7, step=0.01,
                                                 description='Base Payout', readout_format='.2f', layout=Layout(width='300px'))
    target_max_payout_ratio_w = widgets.FloatSlider(value=0.40, min=0.2, max=0.7, step=0.01,
                                                   description='Target Max', readout_format='.2f', layout=Layout(width='300px'))
    cac_per_challenge_w = widgets.FloatText(value=27.5, description='CAC', layout=Layout(width='200px'))
    avg_ltv_challenges_w = widgets.FloatText(value=2.0, description='LTV (chgs)', layout=Layout(width='200px'))
    avg_rescue_price_w = widgets.FloatText(value=350.0, description='Rescue $', layout=Layout(width='200px'))
    rev_share_rate_w = widgets.FloatSlider(value=0.40, min=0.0, max=0.8, step=0.05,
                                          description='Rev Share', readout_format='.2f', layout=Layout(width='300px'))
    n_sims_w = widgets.IntText(value=4000, description='Sims', layout=Layout(width='200px'))

    # Monte Carlo ranges (triangular)
    adoption_low_w = widgets.FloatText(value=0.05, description='Adopt low', layout=Layout(width='160px'))
    adoption_mode_w = widgets.FloatText(value=0.20, description='Adopt mode', layout=Layout(width='160px'))
    adoption_high_w = widgets.FloatText(value=0.35, description='Adopt high', layout=Layout(width='160px'))

    rescues_low_w = widgets.FloatText(value=1.0, description='Resc low', layout=Layout(width='160px'))
    rescues_mode_w = widgets.FloatText(value=2.0, description='Resc mode', layout=Layout(width='160px'))
    rescues_high_w = widgets.FloatText(value=4.0, description='Resc high', layout=Layout(width='160px'))

    cannibal_low_w = widgets.FloatText(value=0.10, description='Cann low', layout=Layout(width='160px'))
    cannibal_mode_w = widgets.FloatText(value=0.30, description='Cann mode', layout=Layout(width='160px'))
    cannibal_high_w = widgets.FloatText(value=0.70, description='Cann high', layout=Layout(width='160px'))

    extra_low_w = widgets.FloatText(value=0.10, description='Extra low', layout=Layout(width='160px'))
    extra_mode_w = widgets.FloatText(value=0.30, description='Extra mode', layout=Layout(width='160px'))
    extra_high_w = widgets.FloatText(value=0.60, description='Extra high', layout=Layout(width='160px'))

    left_col = VBox([
        widgets.HTML('<b>Baseline Business</b>'),
        N_challenges_w,
        challenge_fee_w,
        baseline_payout_ratio_w,
        target_max_payout_ratio_w,
        cac_per_challenge_w,
        avg_ltv_challenges_w,
        avg_rescue_price_w,
        rev_share_rate_w,
        n_sims_w,
    ])

    right_col = VBox([
        widgets.HTML('<b>DRS Behavior Ranges (Triangular)</b>'),
        HBox([adoption_low_w, adoption_mode_w, adoption_high_w]),
        HBox([rescues_low_w, rescues_mode_w, rescues_high_w]),
        HBox([cannibal_low_w, cannibal_mode_w, cannibal_high_w]),
        HBox([extra_low_w, extra_mode_w, extra_high_w]),
    ])

    ui = HBox([left_col, right_col])

    def run_and_plot(
        N_challenges,
        challenge_fee,
        baseline_payout_ratio,
        target_max_payout_ratio,
        cac_per_challenge,
        avg_ltv_challenges,
        avg_rescue_price,
        rev_share_rate,
        n_sims,
        adoption_low,
        adoption_mode,
        adoption_high,
        rescues_low,
        rescues_mode,
        rescues_high,
        cannibal_low,
        cannibal_mode,
        cannibal_high,
        extra_low,
        extra_mode,
        extra_high,
    ):
        # Build param ranges
        adoption_range = (adoption_low, adoption_mode, adoption_high)
        rescues_range = (rescues_low, rescues_mode, rescues_high)
        cannibal_range = (cannibal_low, cannibal_mode, cannibal_high)
        extra_payout_range = (extra_low, extra_mode, extra_high)

        # Run Monte Carlo
        df, summary = simulate_drs(
            n_sims=int(n_sims),
            N_challenges=int(N_challenges),
            challenge_fee=float(challenge_fee),
            baseline_payout_ratio=float(baseline_payout_ratio),
            cac_per_challenge=float(cac_per_challenge),
            avg_ltv_challenges=float(avg_ltv_challenges),
            avg_rescue_price=float(avg_rescue_price),
            target_max_payout_ratio=float(target_max_payout_ratio),
            adoption_range=adoption_range,
            rescues_range=rescues_range,
            cannibal_range=cannibal_range,
            extra_payout_range=extra_payout_range,
            rev_share_rate=float(rev_share_rate),
        )

        # ---- Text summary & verdict ----
        prob_good = summary['prob_good_deal']
        mean_delta = summary['mean_delta_net_profit']
        mean_payout = summary['mean_payout_ratio']
        baseline_ltv = summary['baseline_ltv_profit']
        mean_ltv = summary['mean_ltv_profit']

        if prob_good >= 0.7 and mean_delta > 0:
            verdict = 'LIKELY GOOD DEAL'
        elif prob_good >= 0.4 and mean_delta > -0.05 * abs(summary['baseline_net_profit']):
            verdict = 'MARGINAL – NEGOTIATE TERMS / LOWER REV SHARE'
        else:
            verdict = 'LIKELY BAD DEAL AT THESE TERMS'

        print('--- Summary (per cohort) ---')
        print(f"Mean Δ net profit: {mean_delta:,.0f}")
        print(f"Mean payout ratio: {mean_payout:.3f} (target ≤ {target_max_payout_ratio:.3f})")
        print(f"Baseline LTV profit: {baseline_ltv:,.2f}")
        print(f"Mean LTV profit with DRS: {mean_ltv:,.2f}")
        print(f"Probability DRS is a 'good deal': {prob_good:.1%}")
        print(f"Verdict: {verdict}\n")

        display(pd.DataFrame([summary]))

        # ---- Charts ----
        # Chart 1: Payout ratio distribution
        print('Chart 1 – Payout ratio distribution: how often DRS pushes you above or below your target payout band.')
        fig, ax = plt.subplots(figsize=(6,4))
        ax.hist(df['payout_ratio'], bins=40, alpha=0.8)
        ax.axvline(target_max_payout_ratio, color='red', linestyle='--', label='Target max')
        ax.set_title('Distribution of Payout Ratio with DRS')
        ax.set_xlabel('Payout ratio')
        ax.set_ylabel('Frequency')
        ax.legend()
        plt.show()

        # Chart 2: Δ net profit distribution
        print('Chart 2 – Change in net profit per cohort relative to baseline (DRS – baseline).')
        fig, ax = plt.subplots(figsize=(6,4))
        ax.hist(df['delta_net_profit'], bins=40, alpha=0.8)
        ax.axvline(0, color='red', linestyle='--', label='Break-even')
        ax.set_title('Δ Net Profit per Cohort (DRS - Baseline)')
        ax.set_xlabel('Δ Net profit')
        ax.set_ylabel('Frequency')
        ax.legend()
        plt.show()

        # Chart 3: Δ LTV distribution
        print('Chart 3 – Change in LTV profit per buyer (DRS – baseline).')
        fig, ax = plt.subplots(figsize=(6,4))
        ax.hist(df['delta_ltv_profit'], bins=40, alpha=0.8)
        ax.axvline(0, color='red', linestyle='--', label='Baseline LTV')
        ax.set_title('Δ LTV Profit per Buyer (DRS - Baseline)')
        ax.set_xlabel('Δ LTV profit')
        ax.set_ylabel('Frequency')
        ax.legend()
        plt.show()

        # Chart 4: DRS revenue distribution
        print('Chart 4 – Distribution of DRS revenue per cohort.')
        fig, ax = plt.subplots(figsize=(6,4))
        ax.hist(df['drs_revenue'], bins=40, alpha=0.8)
        ax.set_title('DRS Revenue Distribution')
        ax.set_xlabel('DRS revenue per cohort')
        ax.set_ylabel('Frequency')
        plt.show()

        # Chart 5: Lost challenge revenue distribution
        print('Chart 5 – Cannibalization: lost challenge revenue due to traders choosing DRS instead of buying new challenges.')
        fig, ax = plt.subplots(figsize=(6,4))
        ax.hist(df['lost_challenge_revenue'], bins=40, alpha=0.8)
        ax.set_title('Lost Challenge Revenue (Cannibalization)')
        ax.set_xlabel('Lost revenue per cohort')
        ax.set_ylabel('Frequency')
        plt.show()

        # Chart 6: Adoption vs Δ net profit
        print('Chart 6 – Relationship between adoption rate and Δ net profit. Shows whether higher adoption helps or hurts.')
        fig, ax = plt.subplots(figsize=(6,4))
        ax.scatter(df['adoption_rate'], df['delta_net_profit'], alpha=0.3, s=10)
        ax.set_title('Adoption vs Δ Net Profit')
        ax.set_xlabel('Adoption rate')
        ax.set_ylabel('Δ Net profit')
        plt.show()

        # Chart 7: Cannibalization vs Δ net profit
        print('Chart 7 – Relationship between cannibalization and Δ net profit. Higher cannibalization usually hurts.')
        fig, ax = plt.subplots(figsize=(6,4))
        ax.scatter(df['cannibalization_rate'], df['delta_net_profit'], alpha=0.3, s=10)
        ax.set_title('Cannibalization vs Δ Net Profit')
        ax.set_xlabel('Cannibalization rate')
        ax.set_ylabel('Δ Net profit')
        plt.show()

        # Chart 8: Extra payout factor vs payout ratio
        print('Chart 8 – How sensitive payout ratio is to the incremental payout factor of DRS.')
        fig, ax = plt.subplots(figsize=(6,4))
        ax.scatter(df['extra_payout_per_drs_rev'], df['payout_ratio'], alpha=0.3, s=10)
        ax.set_title('Extra Payout Factor vs Payout Ratio')
        ax.set_xlabel('Extra payout per $1 of DRS revenue')
        ax.set_ylabel('Payout ratio')
        plt.show()

        # Chart 9: Adoption vs payout ratio
        print('Chart 9 – Adoption rate vs payout ratio (does heavier DRS usage push you above 40%?).')
        fig, ax = plt.subplots(figsize=(6,4))
        ax.scatter(df['adoption_rate'], df['payout_ratio'], alpha=0.3, s=10)
        ax.set_title('Adoption vs Payout Ratio')
        ax.set_xlabel('Adoption rate')
        ax.set_ylabel('Payout ratio')
        plt.show()

        # Chart 10: Rescues vs Δ net profit
        print('Chart 10 – Rescues per adopter vs Δ net profit. Captures impact of heavy repeat rescuers.')
        fig, ax = plt.subplots(figsize=(6,4))
        ax.scatter(df['rescues_per_adopter'], df['delta_net_profit'], alpha=0.3, s=10)
        ax.set_title('Rescues per Adopter vs Δ Net Profit')
        ax.set_xlabel('Rescues per adopter')
        ax.set_ylabel('Δ Net profit')
        plt.show()

        # Chart 11: Payout ratio by deal outcome
        print('Chart 11 – Payout ratio distribution for good vs bad outcome simulations.')
        fig, ax = plt.subplots(figsize=(6,4))
        df.boxplot(column='payout_ratio', by='good_deal', ax=ax)
        ax.set_title('Payout Ratio by Deal Outcome (1=Good, 0=Bad)')
        ax.set_xlabel('Good deal indicator')
        ax.set_ylabel('Payout ratio')
        plt.suptitle('')
        plt.show()

        # Chart 12: Revenue-share sensitivity
        print('Chart 12 – How changing the revenue-share rate to Arizet affects P(good deal) and mean Δ net profit.')
        share_grid = np.linspace(0.0, 0.6, 7)
        sweep_df = revenue_share_sweep(
            share_grid,
            n_sims=max(1000, int(n_sims/4)),
            N_challenges=int(N_challenges),
            challenge_fee=float(challenge_fee),
            baseline_payout_ratio=float(baseline_payout_ratio),
            cac_per_challenge=float(cac_per_challenge),
            avg_ltv_challenges=float(avg_ltv_challenges),
            avg_rescue_price=float(avg_rescue_price),
            target_max_payout_ratio=float(target_max_payout_ratio),
            adoption_range=adoption_range,
            rescues_range=rescues_range,
            cannibal_range=cannibal_range,
            extra_payout_range=extra_payout_range,
        )
        display(sweep_df)

        fig, ax1 = plt.subplots(figsize=(6,4))
        x_vals = sweep_df['rev_share_rate'].to_numpy()
        y_prob = sweep_df['prob_good_deal'].to_numpy()
        y_delta = sweep_df['mean_delta_net_profit'].to_numpy()

        ax1.plot(x_vals, y_prob, marker='o', label='P(good deal)')
        ax1.set_xlabel('Revenue share rate to Arizet')
        ax1.set_ylabel('P(good deal)')
        ax1.set_ylim(0, 1)

        ax2 = ax1.twinx()
        ax2.plot(x_vals, y_delta, color='orange', marker='s', label='Mean Δ net')
        ax2.set_ylabel('Mean Δ net profit')

        ax1.set_title('Revenue Share Sensitivity')
        fig.legend(loc='upper right', bbox_to_anchor=(0.9, 0.9))
        plt.show()

    controls = {
        'N_challenges': N_challenges_w,
        'challenge_fee': challenge_fee_w,
        'baseline_payout_ratio': baseline_payout_ratio_w,
        'target_max_payout_ratio': target_max_payout_ratio_w,
        'cac_per_challenge': cac_per_challenge_w,
        'avg_ltv_challenges': avg_ltv_challenges_w,
        'avg_rescue_price': avg_rescue_price_w,
        'rev_share_rate': rev_share_rate_w,
        'n_sims': n_sims_w,
        'adoption_low': adoption_low_w,
        'adoption_mode': adoption_mode_w,
        'adoption_high': adoption_high_w,
        'rescues_low': rescues_low_w,
        'rescues_mode': rescues_mode_w,
        'rescues_high': rescues_high_w,
        'cannibal_low': cannibal_low_w,
        'cannibal_mode': cannibal_mode_w,
        'cannibal_high': cannibal_high_w,
        'extra_low': extra_low_w,
        'extra_mode': extra_mode_w,
        'extra_high': extra_high_w,
    }

    out = widgets.interactive_output(run_and_plot, controls)
    display(ui, out)


Output()